In [1]:
import pandas as pd
import numpy as np

In [2]:
Std_normal = pd.read_csv('Std_list_normal_2016_se1.csv')
Std_atrisk = pd.read_csv('Std_list_atRist_2016_se1.csv')
Std_all = pd.concat([Std_normal, Std_atrisk])
print(Std_all.shape)
Std_all.head()

(14796, 2)


,MASKED_STUDENT_ID,CUM_GPA
0,LQUYAZCO,3.67
1,GDJS7QSE,2.31
2,F7UUSVKE,3.04
3,M6CJWGRJ,2.66
4,VDEBUG5Z,2.48


In [3]:
All_features = pd.read_csv('2016_se1_lib_lms.csv')
All_features.columns

Index(['MASKED_STUDENT_ID', 'workday', 'weekend', 'morning', 'afternoon',
       'evening', 'overnight', 'workday_ExamMonth', 'weekend_ExamMonth',
       'morning_ExamMonth', 'afternoon_ExamMonth', 'evening_ExamMonth',
       'overnight_ExamMonth', 'workday_notExamMonth', 'weekend_notExamMonth',
       'morning_notExamMonth', 'afternoon_notExamMonth',
       'evening_notExamMonth', 'overnight_notExamMonth', 'workday_firstMonth',
       'weekend_firstMonth', 'morning_firstMonth', 'afternoon_firstMonth',
       'evening_firstMonth', 'overnight_firstMonth', 'examMonth',
       'notExamMonth', 'firstMonth', 'total_checkin', 'label_atRist',
       '09LOGIN_ATTEMPT', '09SESSION_TIMEOUT', '09LOGOUT', '09group', '09db',
       '09myinfo', '09course', '09journal', '09email', '09staff',
       '09annoucements', '09content', '09grade', '10LOGIN_ATTEMPT',
       '10SESSION_TIMEOUT', '10LOGOUT', '10group', '10db', '10myinfo',
       '10course', '10journal', '10email', '10staff', '10annoucements',
 

In [4]:
LMS_features_splited_month = All_features[[
    'MASKED_STUDENT_ID',
    '09LOGIN_ATTEMPT', '09SESSION_TIMEOUT', '09LOGOUT', '09group', '09db',
       '09myinfo', '09course', '09journal', '09email', '09staff',
       '09annoucements', '09content', '09grade', '10LOGIN_ATTEMPT',
       '10SESSION_TIMEOUT', '10LOGOUT', '10group', '10db', '10myinfo',
       '10course', '10journal', '10email', '10staff', '10annoucements',
       '10content', '10grade', '11LOGIN_ATTEMPT', '11SESSION_TIMEOUT',
       '11LOGOUT', '11group', '11db', '11myinfo', '11course', '11journal',
       '11email', '11staff', '11annoucements', '11content', '11grade',
       '12LOGIN_ATTEMPT', '12SESSION_TIMEOUT', '12LOGOUT', '12group', '12db',
       '12myinfo', '12course', '12journal', '12email', '12staff',
       '12annoucements', '12content', '12grade'
]]

In [5]:
LMS_features_merge = pd.DataFrame(LMS_features_splited_month['MASKED_STUDENT_ID'])
features = [
    'LOGIN_ATTEMPT', 'SESSION_TIMEOUT', 'LOGOUT', 'group', 'db',
    'myinfo', 'course', 'journal', 'email', 'staff',
    'annoucements', 'content', 'grade'
]
for f in features:
    LMS_features_merge[f] = LMS_features_splited_month['09'+ f] + LMS_features_splited_month['10'+ f] + LMS_features_splited_month['11'+ f] + LMS_features_splited_month['12'+ f] 

LMS_features_merge.head()

,MASKED_STUDENT_ID,LOGIN_ATTEMPT,SESSION_TIMEOUT,LOGOUT,group,db,myinfo,course,journal,email,staff,annoucements,content,grade
0,8TMIKVZ5,237.0,231.0,6.0,324.0,170.0,64.0,25.0,140.0,1.0,8.0,157.0,1465.0,22.0
1,N2YYKTMZ,168.0,161.0,6.0,6.0,9.0,40.0,6.0,0.0,0.0,7.0,62.0,828.0,29.0
2,BZRW4GD3,87.0,67.0,20.0,6.0,5.0,8.0,8.0,0.0,0.0,5.0,15.0,324.0,3.0
3,HJTBF62Q,77.0,31.0,2.0,0.0,3.0,2.0,0.0,0.0,0.0,3.0,2.0,249.0,0.0
4,33XUIDIG,90.0,81.0,9.0,2.0,2.0,6.0,0.0,0.0,0.0,9.0,25.0,287.0,8.0


In [6]:
## merge with GPA

LMS_features_merge_mtx = pd.merge(Std_all, LMS_features_merge, on='MASKED_STUDENT_ID', how='left').fillna(0)
print(LMS_features_merge_mtx.shape)
print('')

LMS_features_splited_month_mtx = pd.merge(Std_all, LMS_features_splited_month, on='MASKED_STUDENT_ID', how='left').fillna(0)
print(LMS_features_splited_month_mtx.shape)

(14796, 15)

(14796, 54)


In [7]:
LMS_features_merge_mtx.to_csv('2016_se1_LMS_features_with_GPA.csv', index=False)

In [35]:
## Regression with GBDT
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

student_features = LMS_features_merge_mtx ## can be replaced with LMS_features_splited_month_mtx(the next block)

df_features = student_features[[i for i in student_features.columns if i != 'CUM_GPA' and i != 'MASKED_STUDENT_ID']]
df_labels = pd.DataFrame(student_features['CUM_GPA'])

X_train, X_test, y_train, y_test = \
    train_test_split(df_features, df_labels, test_size=0.2)

gbr = GBR()
gbr.fit(X_train, y_train)
gbr_preds = gbr.predict(X_test)
score = mean_squared_error(y_test, gbr_preds)
print(score)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.19088435560669134


In [36]:
## Regression with GBDT
from sklearn.ensemble import GradientBoostingRegressor as GBR
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

student_features = LMS_features_splited_month_mtx 

df_features = student_features[[i for i in student_features.columns if i != 'CUM_GPA' and i != 'MASKED_STUDENT_ID']]
df_labels = pd.DataFrame(student_features['CUM_GPA'])

X_train, X_test, y_train, y_test = \
    train_test_split(df_features, df_labels, test_size=0.2)

gbr = GBR()
gbr.fit(X_train, y_train)
gbr_preds = gbr.predict(X_test)
score = mean_squared_error(y_test, gbr_preds)
print(score)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.1829144408509654
